<a href="https://colab.research.google.com/github/bahramkhanlarov/SQL-Database-Agent-with-LangChain/blob/main/SQL_Database_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
!pip install langchain
!pip install -U langchain-community
!pip install langchain-google-genai
!pip install google-generativeai

In [35]:
# Import Packages

from langchain.utilities import SQLDatabase
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain_google_genai import GoogleGenerativeAI

In [36]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')


**Instantiate the Database**

Before we can instantiate the database we need to go to the SQLite website and download the Chinnock sample database. You can download it here:https://www.sqlitetutorial.net/sqlite-sample-database/. Once download, save it to the same directory as your notebook.

In [37]:
db = SQLDatabase.from_uri("sqlite:///chinook.db")
# check that the database has been instantiated correctly

db.get_usable_table_names()






['albums',
 'artists',
 'customers',
 'employees',
 'genres',
 'invoice_items',
 'invoices',
 'media_types',
 'playlist_track',
 'playlists',
 'tracks']

**Instantiate the LLM**

In [38]:
llm = GoogleGenerativeAI(google_api_key=GOOGLE_API_KEY, model="gemini-pro")

**Create the Agent**

In [39]:
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION)

In [40]:
FORMAT_INSTRUCTIONS = """
Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question"""

**Test the Agent**

Example 1 — Running Queries

In this example I demonstrate how the agent can query a database to answer questions.

In [47]:
agent_executor.run(
    "How many employees are there?"
)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: albums, artists, customers, employees, genres, invoice_items, invoices, media_types, playlist_track, playlists, tracks The most relevant table is employees.
Action: sql_db_schema
Action Input: employees
CREATE TABLE employees (
	"EmployeeId" INTEGER NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"FirstName" NVARCHAR(20) NOT NULL, 
	"Title" NVARCHAR(30), 
	"ReportsTo" INTEGER, 
	"BirthDate" DATETIME, 
	"HireDate" DATETIME, 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60), 
	PRIMARY KEY ("EmployeeId"), 
	FOREIGN KEY("ReportsTo") REFERENCES employees ("EmployeeId")
)

/*
3 rows from employees table:
EmployeeId	LastName	FirstName	Title	ReportsTo	BirthDate	HireDate	Address	City	State	Country	PostalCode	Phone	Fax	Email
1	Adams	Andrew	General Manager	None	1962-0

'8'

**Example 2— Describing a Table**

In this example I demonstrate how the agent can describe a table in the database.

In [48]:
agent_executor.run("Describe the playlist_track table")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: albums, artists, customers, employees, genres, invoice_items, invoices, media_types, playlist_track, playlists, tracksThe most relevant table is playlist_track
Action: sql_db_schema
Action Input: playlist_track
CREATE TABLE playlist_track (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES tracks ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES playlists ("PlaylistId")
)

/*
3 rows from playlist_track table:
PlaylistId	TrackId
1	3402
1	3389
1	3390
*/ The playlist_track table contains information about the playlists and the tracks that are included in them.
Final Answer: The playlist_track table contains two columns: PlaylistId and TrackId. The PlaylistId column references the PlaylistId column in the playlists table, and the TrackId column references the TrackId column in the tracks table. The primary key of th

'The playlist_track table contains two columns: PlaylistId and TrackId. The PlaylistId column references the PlaylistId column in the playlists table, and the TrackId column references the TrackId column in the tracks table. The primary key of the playlist_track table is the combination of the PlaylistId and TrackId columns.'

**Example 3— Recovering from an Error**

In this example, the agent is able to recover from an error after initially trying to access an attribute that doesn’t exist.

In [50]:
agent_executor.run("Who are the top 5 best selling artists?")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: albums, artists, customers, employees, genres, invoice_items, invoices, media_types, playlist_track, playlists, tracksArtists is the most relevant table.
Action: sql_db_schema
Action Input: artists
CREATE TABLE artists (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from artists table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/ I need to query the database to get the top 5 best selling artists.  I should join the artists and albums tables on the ArtistId and then join that result set with the invoice_items table on the AlbumId.  Finally, I should group the results by the artist names and sum the quantity of each album sold.
Action: sql_db_query_checker
Action Input: SELECT artists.Name, SUM(invoice_items.Quantity) AS TotalAlbumsSold
FROM artists
JOIN albums ON artists.ArtistId = albums.ArtistId
JOIN invoice_items ON albums.AlbumId = invoice_items.Albu

'The top 5 best selling artists are: Iron Maiden, Led Zeppelin, Deep Purple, U2, and Metallica.'